<a href="https://colab.research.google.com/github/jini1651/PyTorch/blob/master/IMDB_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pytorch-lightning

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data, datasets

import pytorch_lightning as pl

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
torch.manual_seed(777)                                      
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
pl.seed_everything(7)

7

In [5]:
#각 문장의 전처리 과정 지정
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [ ]:
#data load
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [7]:
#단어 집합 생성
TEXT.build_vocab(trainset, min_freq=10)
LABEL.build_vocab(trainset)

In [8]:
#train, validation set 분리
trainset, valset = trainset.split(split_ratio=0.8)
#단어를 인덱스로 지정, 배치 처리
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset), batch_size=50, shuffle=True, repeat=False)

In [ ]:
vocab_size  = len(TEXT.vocab)

In [ ]:
def prepare_data(self):
    TEXT = data.Field(sequential=True, batch_first=True, lower=True)
    LABEL = data.Field(sequential=False, batch_first=True)
          
    trainset, testset = datasets.IMDB.splits(TEXT, LABEL)
        
    TEXT.build_vocab(trainset, min_freq=10)
    LABEL.build_vocab(trainset)
    
  def train_dataloader(self):
    return DataLoader(self.mnist_train, batch_size=64)
    
  def val_dataloader(self):
    return DataLoader(self.mnist_val, batch_size=64)
    
  def test_dataloader(self):
    return DataLoader(self,mnist_test, batch_size=64)

In [9]:
class IMDB(pl.LightningModule):

    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes):
        super(IMDB, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.rnn = nn.LSTM(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.FC = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        x, (_status,c_n) = self.rnn(x)
        h_t = x[:,-1,:] 
        output = self.FC(h_t) 
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch.text, batch.label
        y.data.sub_(1)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        result = pl.TrainResult(loss)
        result.log('train_loss', loss, on_epoch=True)
        return result
        
    def validation_step(self, batch, batch_idx):
        x, y = batch.text, batch.label
        y.data.sub_(1)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss)
        return result

    def test_step(self, batch, batch_idx):
        x, y = batch.text, batch.label
        y.data.sub_(1)
        y_hat = self(x)
        accuracy = (torch.argmax(y_hat, 1) == y).float().mean()
        loss = F.cross_entropy(y_hat, y)
        result = pl.EvalResult()
        result.log('test_accuracy', accuracy)
        return result

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [11]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

<IPython.core.display.Javascript object>

In [12]:
model = IMDB(1, 200, vocab_size, 100, 2)
trainer = pl.Trainer(gpus=1, max_epochs=50) 
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name  | Type      | Params
------------------------------------
0 | embed | Embedding | 2 M   
1 | rnn   | LSTM      | 241 K 
2 | FC    | Linear    | 402   


Saving latest checkpoint..


1

In [13]:
trainer.test(model, test_dataloaders=test_iter)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.8596, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_accuracy': 0.8595600128173828}]

In [14]:
r

NameError: ignored